# Mini-Project - Analysis of consumer buying behavior

<font color='yellow'>Project description:</font>
* The project consisted of analyzing purchase data from a web site;
* The data were provided in JSON format, and were collected from product purchases made online, with information on product type, age, gender, purchase value, among others;
* The goal was to deliver an analysis of consumer buying behavior, which could provide future support to feed machine learning models and make predictions about future behaviors.

<font color='yellow'>Directory for the database:</font> displayed in the project folder.

<font color='yellow'>Languages used for this project:</font> Python.

In [1]:
from platform import python_version
print('Python language version used in this project:', python_version())

Python language version used in this project: 3.10.3


In [2]:
import pandas as pd
import numpy as np
import statistics as sts
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings('ignore')

In [3]:
loadFile = "purchaseData.json"
purchaseFile = pd.read_json(loadFile, orient = "records")
purchaseFile.columns = ['Login', 'Age', 'Gender', 'ItemID', 'Product', 'Cost']
purchaseFile.loc[purchaseFile['Gender'] == 'Masculino', 'Gender'] = 'Male'
purchaseFile.loc[purchaseFile['Gender'] == 'Feminino', 'Gender'] = 'Female'
purchaseFile.loc[purchaseFile['Gender'] == 'Outro / Não Divulgado', 'Gender'] = 'Other'
purchaseFile.head()

,Login,Age,Gender,ItemID,Product,Cost
0,Aelalis34,38,Male,165,Bone Crushing Silver Skewer,3.37
1,Eolo46,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32
2,Assastnya25,34,Male,174,Primitive Blade,2.46
3,Pheusrical25,21,Male,92,Final Critic,1.36
4,Aela59,23,Male,63,Stormfury Mace,1.27


## Information about consumers

In [4]:
# Verification of the need for initial treatment:
purchaseFile.isnull().sum()

Login      0
Age        0
Gender     0
ItemID     0
Product    0
Cost       0
dtype: int64

In [5]:
# Number of unique consumers:
numberOfConsumers = len(purchaseFile.groupby('Login').size())
print(numberOfConsumers)

573


## General analysis of purchases

In [6]:
# Number of unique items by the Product column:
purchaseFile.groupby('Product').size()

Product
Abyssal Shard                      3
Aetherius, Boon of the Blessed     4
Agatha                             5
Alpha                              7
Alpha, Oath of Zeal                7
                                  ..
Wolf                               5
Wolf, Promise of the Moonwalker    7
Worldbreaker                       2
Yearning Crusher                   2
Yearning Mageblade                 4
Length: 179, dtype: int64

In [7]:
# Number of unique items by the ItemID column:
purchaseFile.groupby('ItemID').size()

ItemID
0      1
1      4
2      1
3      1
4      1
      ..
179    7
180    5
181    3
182    5
183    5
Length: 183, dtype: int64

In [8]:
# It is observed that there is an inconsistency in the data that needs to be resolved

# Analysis for when the product has the same nomenclature but the ItemID is different:

tempdf = pd.DataFrame()
productA_Array = []
itemA_Array = []
indexA_Array = []
productB_Array = []
itemB_Array = []
indexB_Array = []

for a in range(len(purchaseFile)):
    for b in range(len(purchaseFile)):
        if purchaseFile['Product'][a] == purchaseFile['Product'][b] and purchaseFile['ItemID'][a] != purchaseFile['ItemID'][b]:
            indexA_Array.append(a)
            productA_Array.append(purchaseFile['Product'][a])
            itemA_Array.append(purchaseFile['ItemID'][a])
            indexB_Array.append(b)
            productB_Array.append(purchaseFile['Product'][b])
            itemB_Array.append(purchaseFile['ItemID'][b])
tempdf['Index_A'] = indexA_Array
tempdf['Product_A'] = productA_Array
tempdf['ItemID_A'] = itemA_Array
tempdf['Index_B'] = indexB_Array
tempdf['Product_B'] = productB_Array
tempdf['ItemID_B'] = itemB_Array
tempdf

,Index_A,Product_A,ItemID_A,Index_B,Product_B,ItemID_B
0,3,Final Critic,92,54,Final Critic,101
1,3,Final Critic,92,193,Final Critic,101
2,3,Final Critic,92,606,Final Critic,101
3,3,Final Critic,92,612,Final Critic,101
4,3,Final Critic,92,663,Final Critic,101
...,...,...,...,...,...,...
201,737,Final Critic,101,509,Final Critic,92
202,737,Final Critic,101,572,Final Critic,92
203,737,Final Critic,101,648,Final Critic,92
204,770,Persuasion,141,26,Persuasion,132


In [9]:
# Analysis for when the ItemID has the same naming but the product naming is different:

tempdf = pd.DataFrame()
productA_Array = []
itemA_Array = []
indexA_Array = []
productB_Array = []
itemB_Array = []
indexB_Array = []

for a in range(len(purchaseFile)):
    for b in range(len(purchaseFile)):
        if purchaseFile['Product'][a] != purchaseFile['Product'][b] and purchaseFile['ItemID'][a] == purchaseFile['ItemID'][b]:
            indexA_Array.append(a)
            productA_Array.append(purchaseFile['Product'][a])
            itemA_Array.append(purchaseFile['ItemID'][a])
            indexB_Array.append(b)
            productB_Array.append(purchaseFile['Product'][b])
            itemB_Array.append(purchaseFile['ItemID'][b])
tempdf['Index_A'] = indexA_Array
tempdf['Product_A'] = productA_Array
tempdf['ItemID_A'] = itemA_Array
tempdf['Index_B'] = indexB_Array
tempdf['Product_B'] = productB_Array
tempdf['ItemID_B'] = itemB_Array
tempdf

,Index_A,Product_A,ItemID_A,Index_B,Product_B,ItemID_B
0,13,"Piety, Guardian of Riddles",76,320,Haunted Bronzed Bludgeon,76
1,13,"Piety, Guardian of Riddles",76,405,Haunted Bronzed Bludgeon,76
2,13,"Piety, Guardian of Riddles",76,488,Haunted Bronzed Bludgeon,76
3,13,"Piety, Guardian of Riddles",76,568,Haunted Bronzed Bludgeon,76
4,320,Haunted Bronzed Bludgeon,76,13,"Piety, Guardian of Riddles",76
5,405,Haunted Bronzed Bludgeon,76,13,"Piety, Guardian of Riddles",76
6,488,Haunted Bronzed Bludgeon,76,13,"Piety, Guardian of Riddles",76
7,568,Haunted Bronzed Bludgeon,76,13,"Piety, Guardian of Riddles",76


In [10]:
# Conclusion: the database has several inconsistencies
# Attempting a different approach by grouping similar products together:
purchaseFile['Product'] = list(purchaseFile['Product'].map(lambda x: [i.strip() for i in x.split(',')]))
for i in range(len(purchaseFile)):
    purchaseFile['Product'][i] = purchaseFile['Product'][i][0]
tempUniqueProduct = purchaseFile.groupby('Product').size()
uniqueProduct = pd.DataFrame()
uniqueProduct['Product'] = list(tempUniqueProduct.index)
uniqueProduct['Number of purchases'] = list(tempUniqueProduct.values)
uniqueProduct.sort_values(['Number of purchases'], ascending = False)

,Product,Number of purchases
3,Alpha,20
28,Darkheart,15
52,Final Critic,14
168,Wolf,12
6,Arcane Gem,11
...,...,...
53,Flux,1
45,Exiled Mithril Longsword,1
156,Verdict,1
37,Downfall,1


In [11]:
# Organization of the 10 best selling products:
topSellingProducts = pd.DataFrame()
topSellingProducts = uniqueProduct.sort_values(['Number of purchases'], ascending = False).head(10)
topSellingProducts

,Product,Number of purchases
3,Alpha,20
28,Darkheart,15
52,Final Critic,14
168,Wolf,12
6,Arcane Gem,11
9,Betrayal,11
132,Stormcaller,10
22,Celeste,10
167,Woeful Adamantite Claymore,9
149,Trickster,9


In [12]:
# Final result with the 10 most sold products, considering the number of sales, the average cost per sale and the total amount collected:
priceAverage = []
totalYield = []
for a in list(topSellingProducts['Product']):
    arrayTemp = []
    for b in range(len(purchaseFile)):
        if purchaseFile['Product'][b] == a:
            arrayTemp.append(purchaseFile['Cost'][b])
    priceAverage.append(round(sts.mean(arrayTemp), 2))
    totalYield.append(sum(arrayTemp))
topSellingProducts['Average cost'] = priceAverage
topSellingProducts['Total Yield'] = totalYield
topSellingProducts

,Product,Number of purchases,Average cost,Total Yield
3,Alpha,20,2.02,40.38
28,Darkheart,15,3.34,50.12
52,Final Critic,14,2.76,38.60
168,Wolf,12,1.86,22.36
6,Arcane Gem,11,2.23,24.53
9,Betrayal,11,2.35,25.85
132,Stormcaller,10,3.47,34.65
22,Celeste,10,3.15,31.50
167,Woeful Adamantite Claymore,9,1.24,11.16
149,Trickster,9,2.07,18.63


## Demographic analysis

In [13]:
demographicAnalysis = purchaseFile.groupby('Gender').size()
dfDemographicAnalysis = pd.DataFrame()
dfDemographicAnalysis['Gender'] = list(demographicAnalysis.index)
dfDemographicAnalysis['Count'] = list(demographicAnalysis.values)
tempArray = []
for i in range(len(dfDemographicAnalysis)):
    tempArray.append(round(dfDemographicAnalysis['Count'][i]/dfDemographicAnalysis['Count'].sum()*100, 2))
dfDemographicAnalysis['Percentage'] = tempArray
dfDemographicAnalysis

,Gender,Count,Percentage
0,Female,136,17.44
1,Male,633,81.15
2,Other,11,1.41


## Analysis of purchases by gender

In [14]:
priceAverage = []
totalYield = []
for a in list(dfDemographicAnalysis['Gender']):
    arrayTemp = []
    for b in range(len(purchaseFile)):
        if purchaseFile['Gender'][b] == a:
            arrayTemp.append(purchaseFile['Cost'][b])
    priceAverage.append(round(sts.mean(arrayTemp), 2))
    totalYield.append(sum(arrayTemp))
dfDemographicAnalysis['Average cost'] = priceAverage
dfDemographicAnalysis['Total Yield'] = totalYield
dfDemographicAnalysis

,Gender,Count,Percentage,Average cost,Total Yield
0,Female,136,17.44,2.82,382.91
1,Male,633,81.15,2.95,1867.68
2,Other,11,1.41,3.25,35.74


In [15]:
# Analysis of age groups:
purchaseFile.groupby('Age').size()

Age
7     19
8      3
9      6
10     4
11     9
12     5
13    11
14     6
15    47
16    23
17    17
18    24
19    22
20    98
21    43
22    68
23    57
24    70
25    67
26    13
27    19
28     5
29    21
30    18
31    16
32    11
33    11
34     8
35    12
36     7
37     9
38     9
39     5
40    14
42     1
43     1
45     1
dtype: int64

In [16]:
# Grouping by age group according to gender:
ageGroup = ['14 years old or younger', 'Between 15 and 23 years old', 'Between 24 and 32 years old', 'Between 33 and 40 years old', \
    'Over 40 years old']
dfAgeGroupByGender = pd.DataFrame()

for a in list(dfDemographicAnalysis['Gender']):
    arrayTemp = []
    for b in range(len(purchaseFile)):
        if purchaseFile['Gender'][b] == a and purchaseFile['Age'][b] <= 14:
            arrayTemp.append(purchaseFile['Cost'][b])
    dfAgeGroupByGender = dfAgeGroupByGender.append({\
        'Gender' : a, \
        'Age Group': ageGroup[0], \
        'Count' : purchaseFile.loc[(purchaseFile['Gender'] == a) & (purchaseFile['Age'] <= 14), 'Login'].count(), \
        'Percentage' : round(purchaseFile.loc[(purchaseFile['Gender'] == a) & (purchaseFile['Age'] <= 14), 'Login'].count()/dfDemographicAnalysis['Count'].sum()*100, 2), \
        'Average Cost' : round(sts.mean(arrayTemp), 2), \
        'Total Yield' : sum(arrayTemp)}, ignore_index = True)
for a in list(dfDemographicAnalysis['Gender']):
    arrayTemp = []
    for b in range(len(purchaseFile)):
        if purchaseFile['Gender'][b] == a and 15 <= purchaseFile['Age'][b] <= 23:
            arrayTemp.append(purchaseFile['Cost'][b])
    dfAgeGroupByGender = dfAgeGroupByGender.append({\
        'Gender' : a, \
        'Age Group': ageGroup[1], \
        'Count' : purchaseFile.loc[(purchaseFile['Gender'] == a) & (purchaseFile['Age'] <= 15) & (purchaseFile['Age'] <= 23), 'Login'].count(), \
        'Percentage' : round(purchaseFile.loc[(purchaseFile['Gender'] == a) & (purchaseFile['Age'] >= 15) & (purchaseFile['Age'] <= 23), 'Login'].count()/dfDemographicAnalysis['Count'].sum()*100, 2), \
        'Average Cost' : round(sts.mean(arrayTemp), 2), \
        'Total Yield' : sum(arrayTemp)}, ignore_index = True)
for a in list(dfDemographicAnalysis['Gender']):
    arrayTemp = []
    for b in range(len(purchaseFile)):
        if purchaseFile['Gender'][b] == a and 24 <= purchaseFile['Age'][b] <= 32:
            arrayTemp.append(purchaseFile['Cost'][b])
    dfAgeGroupByGender = dfAgeGroupByGender.append({\
        'Gender' : a, \
        'Age Group': ageGroup[2], \
        'Count' : purchaseFile.loc[(purchaseFile['Gender'] == a) & (purchaseFile['Age'] >= 24) & (purchaseFile['Age'] <= 32), 'Login'].count(), \
        'Percentage' : round(purchaseFile.loc[(purchaseFile['Gender'] == a) & (purchaseFile['Age'] >= 24) & (purchaseFile['Age'] <= 32), 'Login'].count()/dfDemographicAnalysis['Count'].sum()*100, 2), \
        'Average Cost' : round(sts.mean(arrayTemp), 2), \
        'Total Yield' : sum(arrayTemp)}, ignore_index = True)
for a in list(dfDemographicAnalysis['Gender']):
    arrayTemp = []
    for b in range(len(purchaseFile)):
        if purchaseFile['Gender'][b] == a and 33 <= purchaseFile['Age'][b] <= 40:
            arrayTemp.append(purchaseFile['Cost'][b])
    dfAgeGroupByGender = dfAgeGroupByGender.append({\
        'Gender' : a, \
        'Age Group': ageGroup[3], \
        'Count' : purchaseFile.loc[(purchaseFile['Gender'] == a) & (purchaseFile['Age'] >= 33) & (purchaseFile['Age'] <= 40), 'Login'].count(), \
        'Percentage' : round(purchaseFile.loc[(purchaseFile['Gender'] == a) & (purchaseFile['Age'] >= 33) & (purchaseFile['Age'] <= 40), 'Login'].count()/dfDemographicAnalysis['Count'].sum()*100, 2), \
        'Average Cost' : round(sts.mean(arrayTemp), 2), \
        'Total Yield' : sum(arrayTemp)}, ignore_index = True)
for a in list(dfDemographicAnalysis['Gender']):
    arrayTemp = []
    for b in range(len(purchaseFile)):
        if purchaseFile['Gender'][b] == a and purchaseFile['Age'][b] >= 40:
            arrayTemp.append(purchaseFile['Cost'][b])
    if arrayTemp == []:
        arrayTemp.append(0)
    dfAgeGroupByGender = dfAgeGroupByGender.append({\
        'Gender' : a, \
        'Age Group': ageGroup[4], \
        'Count' : purchaseFile.loc[(purchaseFile['Gender'] == a) & (purchaseFile['Age'] >= 40), 'Login'].count(), \
        'Percentage' : round(purchaseFile.loc[(purchaseFile['Gender'] == a) & (purchaseFile['Age'] >= 40), 'Login'].count()/dfDemographicAnalysis['Count'].sum()*100, 2), \
        'Average Cost' : round(sts.mean(arrayTemp), 2), \
        'Total Yield' : sum(arrayTemp)}, ignore_index = True)
dfAgeGroupByGender

,Gender,Age Group,Count,Percentage,Average Cost,Total Yield
0,Female,14 years old or younger,15,1.92,2.84,42.57
1,Male,14 years old or younger,47,6.03,2.85,133.84
2,Other,14 years old or younger,1,0.13,4.00,4.00
3,Female,Between 15 and 23 years old,19,8.21,2.79,178.74
4,Male,Between 15 and 23 years old,90,42.69,2.90,966.38
5,Other,Between 15 and 23 years old,1,0.26,3.30,6.61
6,Female,Between 24 and 32 years old,42,5.38,2.75,115.58
7,Male,Between 24 and 32 years old,194,24.87,3.06,593.99
8,Other,Between 24 and 32 years old,4,0.51,3.84,15.35
9,Female,Between 33 and 40 years old,15,1.92,3.07,46.02


## Most popular consumers

In [17]:
# Recording a list with unique buyers:
uniquePopularConsumers = list(purchaseFile.groupby('Login').size().index)
uniquePopularConsumers

['Adairialis76',
 'Aduephos78',
 'Aeduera68',
 'Aela49',
 'Aela59',
 'Aelalis34',
 'Aelin32',
 'Aeliriam77',
 'Aeliriarin93',
 'Aeliru63',
 'Aellyria80',
 'Aellyrialis39',
 'Aellysup38',
 'Aelollo59',
 'Aenarap34',
 'Aenasu69',
 'Aeral43',
 'Aeral85',
 'Aeral97',
 'Aeri84',
 'Aerillorin70',
 'Aerithllora36',
 'Aerithnucal56',
 'Aerithnuphos61',
 'Aerithriaphos45',
 'Aesty51',
 'Aesur96',
 'Aethe80',
 'Aethedru70',
 'Aidain51',
 'Aidaira26',
 'Aidaira48',
 'Aiduecal76',
 'Aiduesu83',
 'Ailaesuir66',
 'Aillycal84',
 'Aillyriadru65',
 'Aina42',
 'Aina43',
 'Airal46',
 'Airi27',
 'Airidil41',
 'Airithrin43',
 'Aisur51',
 'Aisurphos78',
 'Aithelis62',
 'Alaephos75',
 'Alaesu77',
 'Alaesu91',
 'Alallo58',
 'Alarap40',
 'Alim85',
 'Alo67',
 'Anallorgue57',
 'Arithllorin55',
 'Assassa38',
 'Assassa43',
 'Assassasda84',
 'Assassasta79',
 'Assastnya25',
 'Assesi91',
 'Assilsan72',
 'Assistasda90',
 'Assistast50',
 'Assithasta65',
 'Assosia38',
 'Assosiasta83',
 'Assossa43',
 'Assylla81',
 'Astyd

In [18]:
# Selecting the top 5 users who spent the most on purchases:
dfPopularConsumers = pd.DataFrame()
for a in uniquePopularConsumers:
    tempdf = purchaseFile.loc[purchaseFile['Login'] == a]
    dfPopularConsumers = dfPopularConsumers.append({\
        'Login' : a, \
        'Purchased Items (ID)' : list(tempdf['ItemID']), \
        'Purchased Items (Product)' : list(tempdf['Product']), \
        'Count' : tempdf['Product'].count(), \
        'Average Cost': round(sts.mean(tempdf['Cost']), 2), \
        'Total Spend' : round(tempdf['Cost'].sum(), 2), \
        'Percentage' : round(tempdf['Cost'].sum()/purchaseFile['Cost'].sum()*100, 2)}, \
        ignore_index = True)
dfPopularConsumers.sort_values('Total Spend', ascending = False).head(5)

,Login,Purchased Items (ID),Purchased Items (Product),Count,Average Cost,Total Spend,Percentage
538,Undirrala66,"[144, 115, 62, 18, 133]","[Blood Infused Guardian, Spectral Diamond Doom...",5,3.41,17.06,0.75
428,Saedue76,"[13, 140, 7, 73]","[Serenity, Striker, Thorn, Ritual Mace]",4,3.39,13.56,0.59
354,Mindimnya67,"[140, 163, 145, 161]","[Striker, Thunderfury Scimitar, Fiery Glass Cr...",4,3.19,12.74,0.56
181,Haellysu29,"[96, 166, 91]","[Blood-Forged Skeletal Spine, Thirsty Iron Rea...",3,4.24,12.73,0.56
120,Eoda93,"[35, 173, 76]","[Heartless Bone Dualblade, Stormfury Longsword...",3,3.86,11.58,0.51


## Most popular items

In [19]:
# Recording a list with unique items by ItemID:
uniquePopularItems = list(purchaseFile.groupby('Product').size().index)
uniquePopularItems

['Abyssal Shard',
 'Aetherius',
 'Agatha',
 'Alpha',
 'Amnesia',
 'Apocalyptic Battlescythe',
 'Arcane Gem',
 'Avenger',
 'Azurewrath',
 'Betrayal',
 'Betrayer',
 'Blade of the Grave',
 'Blazefury',
 'Blazeguard',
 'Blindscythe',
 'Blood Infused Guardian',
 'Blood-Forged Skeletal Spine',
 "Bloodlord's Fetish",
 'Bone Crushing Silver Skewer',
 'Bonecarvin Battle Axe',
 'Brimstone',
 'Brutality Ivory Warmace',
 'Celeste',
 'Chaos',
 'Conqueror Adamantite Mace',
 'Crucifer',
 'Crying Steel Sickle',
 'Curved Axe',
 'Darkheart',
 'Dawn',
 'Dawne',
 'Deadline',
 'Deathraze',
 'Deluge',
 'Demise',
 'Despair',
 'Devine',
 'Downfall',
 "Dragon's Greatsword",
 'Dreamkiss',
 'Dreamsong',
 'Emberling',
 'Endbringer',
 'Eternal Cleaver',
 'Exiled Doomblade',
 'Exiled Mithril Longsword',
 'Expiration',
 'Extraction',
 "Faith's Scimitar",
 'Fate',
 'Feral Katana',
 'Fiery Glass Crusader',
 'Final Critic',
 'Flux',
 'Foul Edge',
 'Foul Titanium Battle Axe',
 "Freak's Bite",
 'Frenzied Scimitar',
 'Fre

In [20]:
# Selecting the top 5 most popular products by number of sales:
dfPopularItems = pd.DataFrame()
for a in uniquePopularItems:
    tempdf = purchaseFile.loc[purchaseFile['Product'] == a]
    dfPopularItems = dfPopularItems.append({\
        'Product' : a, \
        'ItemsID' : list(set(map(lambda x: str(x), list(tempdf['ItemID'])))), \
        'Purchases' : tempdf['Product'].count(), \
        'Average Cost': round(sts.mean(tempdf['Cost']), 2), \
        'Total Spend' : round(tempdf['Cost'].sum(), 2), \
        'Percentage' : round(tempdf['Cost'].sum()/purchaseFile['Cost'].sum()*100, 2)}, \
        ignore_index = True)
dfPopularItems.sort_values('Purchases', ascending = False).head(5)

,Product,ItemsID,Purchases,Average Cost,Total Spend,Percentage
3,Alpha,"[47, 130, 79]",20,2.02,40.38,1.77
28,Darkheart,"[158, 152]",15,3.34,50.12,2.19
52,Final Critic,"[92, 101]",14,2.76,38.60,1.69
168,Wolf,"[60, 179]",12,1.86,22.36,0.98
6,Arcane Gem,[84],11,2.23,24.53,1.07


## Most profitable items

In [21]:
# Same consideration as before, but now the filter is in relation to the 'Total Spend' column:
dfProbitableItems = dfPopularItems.sort_values('Total Spend', ascending = False).head(5)
dfProbitableItems

,Product,ItemsID,Purchases,Average Cost,Total Spend,Percentage
28,Darkheart,"[158, 152]",15,3.34,50.12,2.19
3,Alpha,"[47, 130, 79]",20,2.02,40.38,1.77
52,Final Critic,"[92, 101]",14,2.76,38.60,1.69
107,Retribution Axe,[34],9,4.14,37.26,1.63
132,Stormcaller,"[180, 30]",10,3.47,34.65,1.52
